In [71]:
import FINE as fn
from getData import getData
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
data = getData()

cwd = os.getcwd()
inputDataPath = os.path.join(cwd, "InputData")
data = {}
data = getData()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [72]:
locations = data['locations']
commodityUnitDict = {'electricity': r'kW$_{el}$','hydrogen': r'kW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [73]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='€', lengthUnit='m', verboseLogLevel=0)

In [74]:
esM.add(fn.Source(esM=esM,
                  name='PV',
                  commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], 
                  capacityMax=data['PV, capacityMax'],
                  economicLifetime = 20,
                  investPerCapacity=1400,
                  opexPerCapacity=1400*0.02
                  )   )

In [75]:
Edemand_operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'E_Demand.xlsx'))
data.update({'Electricity demand, operationRateFix': Edemand_operationRateFix})

In [76]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, 
                operationRateFix=data['Electricity demand, operationRateFix']))

In [77]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', 
                      physicalUnit=r'kW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

In [78]:
esM.add(fn.Conversion(esM=esM, name='Fuel Cell', 
                      physicalUnit=r'kW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

In [79]:
esM.add(fn.Storage(esM=esM,
                   name='Battery Storage',
                   commodity='electricity',
                   cyclicLifetime=10000,
                   chargeEfficiency=0.95,
                   dischargeEfficiency=0.95,
                   chargeRate=0.5,
                   dischargeRate=0.5,
                   interestRate = 0.04,
                   economicLifetime = 12,
                   investPerCapacity=700,
                   ))

In [ ]:
esM.optimize(timeSeriesAggregation=False, logFileName='', optimizationSpecs='cuts=0 method=2')

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(5.9697 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(6.0392 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(20.8575 sec)

Declaring shared potential constraint...
		(0.0000 sec)

Declaring commodity balances...
